In [ ]:
# Run setup_imports to enable importing from src/
%run ../src/setup_imports.py

In [ ]:
from src.utils.ibm import get_backend, get_backend_snapshot, run_and_monitor_circuit
from src.utils.python_utils import clean_datetimes
from src.utils.circuits import chsh_circuit, phi_plus_meassured, compute_entanglement_correlation
from qiskit_ibm_runtime import SamplerV2 as Sampler
from datetime import datetime
import json
import time
import pandas as pd

In [ ]:
# Parametrize experiment
reps = 20
introspection_shots = 1024
main_circuit_shots = 1024

In [ ]:
backend = get_backend()

In [ ]:
# Main experiment loop

results = []
for i in range(reps):
    entry = {}
    
    # --- Backend snapshot; need we do this more than once?---
    entry['backend_snapshot'] = get_backend_snapshot(backend)

    # --- Introspection phase ---
    entry['introspection'] = run_and_monitor_circuit(chsh_circuit(), backend, introspection_shots)
    entry['entanglement_score'] = compute_entanglement_correlation(entry['introspection']["counts"])

    # --- Decision phase ---
    if entry['entanglement_score'] > 0.6:  # Arbitrary threshold
        entry['execution'] = run_and_monitor_circuit(phi_plus_meassured(), backend, main_circuit_shots)
    
    results.append(clean_datetimes(entry))
    time.sleep(3600) # wait an hour before next run 

    # Stop at 7AM latest
    if datetime.now().hour == 7:
        print(f"Breaking loop at {datetime.now().strftime('%H:%M:%S')}")
        break
    
# Save results
with open("experiment_results.json", "w") as f:
    json.dump(results, f, indent=2)

# Tabular summary
df = pd.DataFrame([{
    'introspection_start': r.get('introspection', {}).get('timestamps', {}).get('SUBMITTED'),
    'introspection_end': r.get('introspection', {}).get('timestamps', {}).get('DONE'),
    'execution_start': r.get('execution', {}).get('timestamps', {}).get('SUBMITTED'),
    'execution_end': r.get('execution', {}).get('timestamps', {}).get('DONE'),
    'entanglement_score': r.get('entanglement_score'),
} for r in results])
df.to_csv("experiment_summary.csv", index=False)